Практическая работа 4
Автор: Воложанин В.О. 

main.py

In [ ]:
import pygame, sys
import numpy as np
import graphics as g
import math as m
import transform as t
from pygame.locals import *

rf = g.ReferenceFrame(g.Origin(20, 500), g.Unit(75), g.Unit(75))
d = g.Drawer(800, 600, 32, rf)
d.initialize()
d.color = (0, 0, 0)
d.draw_axes(-0.4, 9.0, -0.4, 8.0)

L = np.matrix([[2.0, 0.5, 1], [8.0, 0.5, 1], [8.0, 6.5, 1], [2.0, 6.5, 1]])

xc = (2.0 + 8.0) / 2
yc = (0.5 + 6.5) / 2

d.color = (60, 115, 20)
d.draw_polygon(L, 2)
angle = m.pi / 64
scale = 0.95
L_last = L
for k in range(0, 50):
  L_last = L_last * t.translate(-xc, -yc) * t.scale(scale, scale) * t.rotate(angle) * t.translate(xc, yc)
  d.draw_polygon(L_last, 1)

FPS = 30
clock = pygame.time.Clock()
while True:
  for event in pygame.event.get():
    if event.type == QUIT:
      pygame.quit()
      sys.exit()
    clock.tick(FPS)
    pygame.display.update()


graphics.py

In [ ]:
import pygame

from numpy import matrix


class Origin:

  def __init__(self, x0: int, y0: int):
    self.__x0 = x0
    self.__y0 = y0

  @property
  def x0(self):
    return self.__x0

  @property
  def y0(self):
    return self.__y0


class Unit:

  def __init__(self, pixels: int):
    self.__pixels = pixels

  @property
  def pixels(self):
    return self.__pixels


class ReferenceFrame:

  def __init__(self, origin: Origin, ux: Unit, uy: Unit):
    self.__origin = origin
    self.__unit_x = ux
    self.__unit_y = uy

  @property
  def origin(self):
    return self.__origin

  @property
  def unit_x(self):
    return self.__unit_x

  @property
  def unit_y(self):
    return self.__unit_y


def get_x(rf: ReferenceFrame, x: float):
  scale_x = rf.unit_x.pixels
  x0 = rf.origin.x0
  return int(scale_x * x + x0)


def get_y(rf: ReferenceFrame, y: float):
  scale_y = rf.unit_y.pixels
  y0 = rf.origin.y0
  return int(-scale_y * y + y0)


def get_length(unit: Unit, dl: float):
  scale = unit.pixels
  return int(scale * dl)


class Drawer:

  def __init__(self, res_x: int, res_y: int, color_depth: int,
               rf: ReferenceFrame):
    self.rf = rf
    self.__color = (0, 0, 0)
    self.resolution = (res_x, res_y)
    self.color_depth = color_depth
    self.screen = None
    self.font = None

  def initialize(self):
    pygame.init()
    screen = pygame.display.set_mode(self.resolution, 0, self.color_depth)
    pygame.font.init()
    font = pygame.font.SysFont('Arial', 24)
    self.screen = screen
    self.font = font
    self.screen.fill((255, 255, 255))

  def draw_line(self, x1: float, y1: float, x2: float, y2: float, width: int):
    x1 = get_x(self.rf, x1)
    y1 = get_y(self.rf, y1)
    x2 = get_x(self.rf, x2)
    y2 = get_y(self.rf, y2)
    pygame.draw.line(self.screen, self.color, (x1, y1), (x2, y2), width)

  def draw_text(self, x: float, y: float, text: str):
    x = get_x(self.rf, x)
    y = get_y(self.rf, y)
    text_surface = self.font.render(text, False, self.color)
    self.screen.blit(text_surface, (x, y))

  def draw_polygon(self, points: matrix, width: int):
    n_rows = points.shape[0]
    row_points = []
    for i in range(0, n_rows):
      row_point = (get_x(self.rf, points[i, 0]), get_y(self.rf, points[i, 1]))
      row_points.append(row_point)
    pygame.draw.polygon(self.screen, self.color, row_points, width)

  def draw_axes(self, x_min: float, x_max: float, y_min: float, y_max: float):
    self.draw_line(x_min, 0.0, x_max, 0.0, 2)
    self.draw_line(0.0, y_min, 0.0, y_max, 2)
    self.draw_text(-0.2, -0.1, "0")


transform.py

In [ ]:
import numpy as np
import math as m


def rotate(alpha: float):
  cos_a = m.cos(alpha)
  sin_a = m.sin(alpha)
  return np.matrix([[cos_a, sin_a, 0], [-sin_a, cos_a, 0], [0, 0, 1]])


def translate(dx: float, dy: float):
  return np.matrix([[1, 0, 0], [0, 1, 0], [dx, dy, 1]])


def scale(Sx: float, Sy: float):
  return np.matrix([[Sx, 0, 0], [0, Sy, 0], [0, 0, 1]])
